### Importing all the required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Open Source Computer Vision Library
import cv2 

# keeps track of the objects it has already serialized, so that later references to the same object won’t be serialized again
import pickle

# provides a portable way of using operating system dependent functionality. 
import os
from os import listdir 
from os.path import isfile, join


from sklearn.model_selection import train_test_split # Split arrays or matrices into random train and test subsets
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation

from sklearn.preprocessing import StandardScaler # Standardize features by removing the mean and scaling to unit variance
from sklearn.linear_model import SGDClassifier # Linear classifiers (SVM, logistic regression, a.o.) with SGD training.

from sklearn.metrics import accuracy_score # Accuracy classification score.
from sklearn.metrics import confusion_matrix #Compute confusion matrix to evaluate the accuracy of a classification

### Reading Images from a folder

In [ ]:
mypath='C:/Users/V Sundar/Downloads'# change the path to read corresponding folders.
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)
resized_image = np.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    
    images[n] = cv2.imread( join(mypath,onlyfiles[n]) ) # Reading all the images from folder and storing in numpy array.
    resized_image[n] = cv2.resize(images[n], (100,100)) # Resizing all the images into 100X100 pixels.
    

In [ ]:
print(type(resized_image)) # to check the type.
print(resized_image.shape) # check the no of images present in folder.

### Segmentation by K-Mean Clustering.
K-Means is a least-squares partitioning method that divide a collection of objects into K groups. 

### cv2.kmeans() function in OpenCV for data clustering
#### Input Parameters

1. **samples** : It should be of np.float32 data type, and each feature should be put in a single column.<br>
<br>
2. **nclusters(K)** : Number of clusters required at end<br>
<br>
3. **criteria** : It is the iteration termination criteria. When this criteria is satisfied, algorithm iteration stops. Actually, it should be a tuple of 3 parameters. They are ( type, max_iter, epsilon ):<br>

3.1. type of termination criteria : It has 3 flags as below:<br>
**cv2.TERM_CRITERIA_EPS** - stop the algorithm iteration if specified accuracy, epsilon, is reached. <br>**cv2.TERM_CRITERIA_MAX_ITER** - stop the algorithm after the specified number of iterations, max_iter. <br>
**cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER** - stop the iteration when any of the above condition is met.<br>


3.2. **max_iter** - An integer specifying maximum number of iterations.<br>

3.3. **epsilon** - Required accuracy

In [ ]:
# Initialize all the arrays as empty.
gray = np.empty(len(images), dtype=object)
Y = np.empty(len(images), dtype=object)
label = np.empty(len(images), dtype=object)
res = np.empty(len(images), dtype=object)
res2 = np.empty(len(images), dtype=object)


There are 3 features, say, R,G,B. So we need to reshape the image to an array of Mx3 size (M is number of pixels in image). And after the clustering, we apply centroid values (it is also R,G,B) to all pixels, such that resulting image will have specified number of colors. And again we need to reshape it back to the shape of original image.

In [ ]:
for i in range(0,n):
    Y[i]=resized_image[i].reshape((-1,3))
    Y[i]= np.float32(Y[i])
    
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K=4 # No of clusters=4

for i in range(0,n):    
    ret,label[i],center=cv2.kmeans(Y[i],K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)# Using cv2.Kmeans() Function
    
center = np.uint8(center) #  array of centers of clusters.

for i in range(0,n):    
    res[i] = center[label[i].flatten()]
    res2[i] = res[i].reshape((resized_image[i].shape))
    gray[i] = cv2.cvtColor(res2[i], cv2.COLOR_BGR2GRAY)


In [ ]:
# Flatten the segmented array.
for i in range(0,n):
        res2[i]=res2[i].flatten()

In [ ]:
# Variants of numpy.stack function to stack so as to make a single array vertically.
abc = np.empty(30000) # 100*100*3
for i in range(961):# Size of folder-1
    abc = np.vstack((abc, res2[i]))
abc  = abc[1:,:]
abc.shape

(961, 30000)

In [ ]:
# Convert ndarray into Dataframe.
df16=pd.DataFrame(data=abc)
df16['label']=16 # Label the images whichever class it may belong to.

Doing the above procedure for 9 folders and appending it into single Dataframe.

In [ ]:
with open('dfc16-24', 'rb') as pickle_file:
    df = pickle.load(pickle_file)# list_of_sent of summary

In [ ]:
df.head()

#### Slicing the dataframe and taking 5000 points for creating model

In [ ]:
X=df.iloc[:5000,:30000].values
y=df.iloc[:5000,30000].values

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
s=StandardScaler()
X=s.fit_transform(X)# Perform column Standardization 

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2) # Split the dataset into train and test as 80-20 split